<a href="https://colab.research.google.com/github/byJohnWesley/GoogleGeminiProjectSAAI/blob/main/SAAI_backup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
%pip -q install google-genai

In [23]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [24]:
!pip install -q google-adk

In [25]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [26]:
##########################################
# --- Agente 1: Coletor de Dados --- #
##########################################
def agente_coletor(topico, data_de_hoje):

    coletor = Agent(
        name="agente_coletor",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente especialista em coleta de dados do mercado financeiro.
        Sua tarefa é utilizar ferramentas de busca, como google_search, para obter informações atualizadas com no máximo um mês de defasagem a partir da data atual.
        A partir do ativo fornecido pelo usuário — que pode ser um nome, setor ou ticker —,
        você deverá coletar e estruturar uma base de dados com ativos relevantes e correlacionados ao setor mencionado ou ao ativo especificado.
        Sua abordagem deve ser quantitativa, racional e sistemática, com foco em fundamentos sólidos.
        Nos critérios de crescimento, avalie lucro líquido crescente, consistência nos resultados, receita crescente, além de margem líquida e EBITDA saudáveis.
        Nos indicadores de endividamento, leve em conta a dívida bruta, dívida líquida, a relação dívida líquida/EBITDA e o índice de cobertura de juros.
        Para a eficiência operacional, observe o giro de ativos e a margem operacional.
        Nos critérios de rentabilidade, analise ROE (retorno sobre o patrimônio líquido) e ROIC (retorno sobre o capital investido).
        Em termos de valuation, utilize múltiplos como P/L (preço/lucro), P/VPA (preço sobre valor patrimonial), EV/EBITDA e dividend yield,
        além de fórmulas clássicas como o Preço Justo (fórmula de Graham) e o Preço Topo (método Bazin), baseando-se sempre em projeções conservadoras de lucro e dividendos.
        Considere também a evolução do patrimônio líquido e a geração de caixa operacional, bem como o CAGR (taxa de crescimento anual composta) da receita, lucro e dividendos.
        A análise deve sempre partir de dados históricos, com projeções prudentes e margem de segurança.
        Com base nesses critérios, monte uma lista de até 45 ativos de destaque, divididos entre 10 ações brasileiras, 10 internacionais, até 10 commodities e até 8 fundos imobiliários,
        e 7 investimentos correlacionados não citados, conforme o setor analisado.
        Para cada ativo, apresente os dados obtidos e justifique sua inclusão com base nos fundamentos levantados.
        Se o input do usuário for um ativo individual e não um setor, direcione a análise para ativos do mesmo setor ou segmento.
        A lista deve conter apenas os melhores ativos cujos dados demonstram o melhor desempenho e consistência segundo os critérios estabelecidos em deterimento a outros ativos.
        """,
        description="Agente que coleta informações e dados sobre empresas no Google",
        tools=[google_search]
    )

    entrada_do_agente_coletor = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    coleta_de_dados = call_agent(coletor, entrada_do_agente_coletor)
    return coleta_de_dados

In [ ]:
########################################################
# --- Agente 2: Analista de Cenário Macroeconômico --- #
########################################################
def agente_analista_macro(topico, coleta_de_dados):
    analista_macro = Agent(
        name="agente_analista_macro",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Analista #################################################
        instruction="""
        Você é um especialista em análise de mercado financeiro com foco em cenários macroeconômicos.
        Sua tarefa é utilizar a ferramenta (google_search) para buscar, coletar e analisar informações macroeconômicas atualizadas — com no máximo um mês de defasagem.
        A partir da lista de empresas, commodities e demais ativos financeiros coletados pelo agente 1, coletor, investigue o contexto macroeconômico que possa afetar diretamente
        ou indiretamente esses ativos. Considere fatores como juros, inflação, PIB, cenário político-econômico nacional e global, política monetária,
        decisões de bancos centrais, conflitos geopolíticos, e outras variáveis relevantes.
        Analise com base em dados históricos, identificando padrões e impactos recorrentes, mas sem deixar que percepções subjetivas ou emoções influenciem sua avaliação.
        Cruze referências confiáveis para selecionar os pontos com maior grau de influência comprovada.
        Relacione como cada um dos ativos analisados pode reagir ao atual contexto macroeconômico, e destaque os possíveis desdobramentos.
        Ao final, para cada ativo avaliado, associe pelo menos duas notícias macroeconômicas relevantes a cada ativo que ajudem a justificar seu raciocínio e enriquecem
        a interpretação do cenário atual. Retorne uma lista clara e justificada com essas análises para os próximos agentes utilizarem.
        """,
        description="Agente que Analisa Macroeconomia e relaciona dados",
        tools=[google_search]
    )

    entrada_do_agente_analista_macro = f"Tópico:{topico}\nDados Coletados: {coleta_de_dados}"
    # Executa o agente
    analise_macro = call_agent(analista_macro, entrada_do_agente_analista_macro)
    return analise_macro

In [ ]:
########################################################
# --- Agente 3: Analista de Cenário Microeconômico --- #
########################################################
def agente_analista_micro(topico, coleta_de_dados):
    analista_micro = Agent(
        name="agente_analista_micro",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Analista Micro #################################################
        instruction="""
        Você é um especialista em análise de mercado financeiro com foco em microeconomia.
        Com base na lista de empresas, commodities e demais ativos financeiros coletados pelo primeiro agente, sua tarefa é utilizar a ferramenta (google_search)
        para buscar, coletar e analisar dados atualizados sobre o cenário microeconômico que possa impactar cada ativo de forma direta ou indireta. Se o ativo analisado
        for extrangeiro, então a analise microeconomica deve ser do cenario onde se encontra aquele ativo.
        Essa análise deve considerar fatores como nível de concorrência no setor, estrutura de mercado, grau de inovação, cadeia de suprimentos e produção,
        regulamentações específicas, barreiras de entrada, estratégias comerciais,
        dinâmica entre fornecedores e distribuidores, custos operacionais e capacidade de repasse de preços ao consumidor.
        Leve em conta a evolução desses fatores ao longo do tempo e como eles historicamente influenciam a performance dos ativos, sempre mantendo uma abordagem racional,
        técnica e livre de vieses emocionais. Cruze diferentes fontes confiáveis usando a ferramenta de busca, identificando padrões,
        fragilidades e forças competitivas. A análise deve também considerar fatores internos das empresas, como decisões estratégicas, mudança de gestão,
        novos contratos, expansão de capacidade, fusões e aquisições, entre outros eventos microeconômicos relevantes.
        Ao final, relacione duas notícias microeconômicas recentes e relevantes para cada ativo analisado,
        justificando sua seleção e detalhando como essas informações afetam o desempenho ou as perspectivas do ativo.
        Apresente suas análises de forma organizada e clara para que o próximo agente possa integrar os resultados.
        """,
        description="Agente que Analisa Microeconomia e relaciona dados",
        tools=[google_search]
    )

    entrada_do_agente_analista_micro = f"Tópico:{topico}\nDados Coletados: {coleta_de_dados}"
    # Executa o agente
    analise_micro = call_agent(analista_micro, entrada_do_agente_analista_micro)
    return analise_micro

In [ ]:
# --- Agente 4: Analista de Cenarios --- #
###########################################

def agente_analista_cenarios(topico, coleta_de_dados, analise_macro, analise_micro):
    analista_cenarios = Agent(
        name="agente_analista_cenarios",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Analista de Cenário #######################################
        instruction="""
Você é um analista especializado em correlacionar dados de micro e macroeconomia, com foco na interpretação de políticas públicas,
dinâmicas de mercado e projeções estratégicas no contexto financeiro. Sua função é integrar todos os dados coletados anteriormente — incluindo os
fundamentos dos ativos, as análises macroeconômicas e microeconômicas — para projetar possíveis desdobramentos sobre o comportamento de cada ativo ao longo do tempo.
Utilizando a ferramenta (google_search), complemente as informações necessárias com atualizações que fortaleçam ou revelem novas sinergias entre os dados analisados.
Você deverá identificar cenários prováveis com base nas políticas econômicas que estão em curso ou em debate (monetárias, fiscais, regulatórias e setoriais),
analisando não apenas os efeitos históricos da implementação dessas medidas, mas também as chances de elas se concretizarem no presente ciclo.
Avalie como essas medidas impactam direta e indiretamente os ativos da lista, buscando entender o jogo de interesses e o equilíbrio de poder entre os atores econômicos
(governo, bancos centrais, empresas, consumidores e mercados internacionais).
A partir disso, estruture três tipos de cenários: o mais provável, o alternativo e o improvável, justificando com dados e lógica de mercado cada projeção.
Para cada ativo, apresente como ele tende a se comportar em cada cenário, destacando os que têm maior resiliência ou propensão a se valorizar independentemente das incertezas.
Melhore a sinergia entre os dados sempre que identificar lacunas ou brechas entre as análises dos agentes anteriores,
elevando o nível de coerência e robustez da projeção.
Por fim, entregue uma lista rankeada dos ativos analisados, priorizando aqueles com melhor posicionamento frente aos cenários projetados,
acompanhada de observações técnicas e estratégicas para os próximos anos.
        """,
        description="Agente analista de Macro e Micro economias, possíveis cenários econômicos e como isso afeta o mercado"
    )

    entrada_do_analista_cenarios = f"""Tópico: {topico}
Dados Coletados: {coleta_de_dados}
Análise Macro: {analise_macro}
Análise Micro: {analise_micro}"""

    # Executa o agente
    analise_final = call_agent(analista_cenarios, entrada_do_analista_cenarios)
    return analise_final


In [ ]:
##############################################
# --- Agente 5: Analista Fundamentalista --- #
##############################################
def agente_analista_fundamentalista(topico, analise_macro, analise_micro, analise_final):
    fundamentalista = Agent(
        name="agente_analista_fundamentalista",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Analista Fundamentalista ##########################################
        instruction="""
   Você é um analista especialista em análise fundamentalista avançada, com uma abordagem quantitativa, racional e sistemática.
   Sua função é consolidar todas as análises anteriores — os fundamentos dos ativos, os dados microeconômicos, macroeconômicos e as projeções de cenários —
   para realizar uma avaliação profunda da qualidade e do potencial de valorização dos ativos selecionados.
   Utilize a ferramenta (google_search) para confirmar e complementar os dados financeiros necessários.
   Baseie-se em uma metodologia sólida que filtre vieses emocionais e permita identificar se um ativo está em “ponto de compra”, “zona neutra” ou “preço topo”,
   sempre fundamentado em histórico, projeções conservadoras e uma margem de segurança adequada. Sua análise deve contemplar múltiplos indicadores estratégicos,
   levando em conta fundamentos operacionais como lucro líquido crescente, consistência nos resultados, receita em expansão, margens líquida e EBITDA saudáveis,
   além de geração de caixa operacional sólida. Os indicadores de endividamento devem incluir dívida bruta, dívida líquida,
   relação dívida líquida/EBITDA e índice de cobertura de juros. Quanto à eficiência, avalie o giro de ativos, a margem operacional e a
   produtividade por segmento. Em relação à rentabilidade, leve em conta o retorno sobre o patrimônio líquido (ROE) e o retorno sobre o capital investido (ROIC).
   No que diz respeito à precificação, utilize múltiplos como P/L, P/VPA, EV/EBITDA e dividend yield, e empregue modelos clássicos como o
   Preço Justo de Graham e o Preço Topo pelo Método Bazin. A análise deve também considerar o balanço patrimonial de cada empresa,
   avaliando a evolução do patrimônio líquido e os indicadores de crescimento, como o CAGR de receita, lucro e dividendos.
   Atribua pesos distintos para cada indicador conforme a natureza do ativo (empresa, commodity, fundo, etc.), e conecte toda essa análise ao cenário político,
   às políticas financeiras em vigor ou previstas, e às condições macro e microeconômicas.
   Com base em todas as análises e dados coletados, selecione os cinco ativos mais sólidos e promissores de cada categoria do setor solicitado,
   priorizando aqueles que, mesmo em cenários adversos, demonstram potencial de valorização robusta, histórica e projetada acima do IPCA, do S&P 500,
   da Nasdaq e de outros benchmarks globais. Certifique-se de que todos os ativos escolhidos atendam aos critérios descritos,
   garantindo máxima confiabilidade e relevância estratégica à sua recomendação final.
   """,
        description="Agente Analista Fundamentalista",
    )

    entrada_do_fundamentalista = f"Tópico: {topico}\nAnalise Macro: {analise_macro}\nAnalise Micro: {analise_micro}\nAnalise Final: {analise_final}"
    # Executa o agente
    fundamentalista = call_agent(fundamentalista, entrada_do_fundamentalista)
    return fundamentalista

In [ ]:
############################################################
# --- Agente 6: Avaliador de Potencial de Investimento --- #
############################################################
def agente_avaliador(topico, analise_macro, analise_micro, analise_final, fundamentalista):
    avaliador = Agent(
        name="agente_avaliador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um analista especialista em análise fundamentalista avançada, com uma abordagem quantitativa, racional e sistemática.
        Sua função é consolidar todas as análises anteriores — fundamentos dos ativos, dados micro e macroeconômicos, e projeções de cenários considerados — para
        realizar uma avaliação profunda da qualidade e do potencial de valorização dos ativos selecionados.
        A partir dos ativos fornecidos — que podem ser nomes, setores ou tickers —, você deverá coletar e estruturar uma base de dados com ativos relevantes e
        correlacionados ao setor mencionado ou ao ativo especificado. Avalie criteriosamente os seguintes indicadores econômicos para cada ativo:
        Crescimento: lucro líquido crescente, consistência nos resultados, receita crescente, margens líquida e EBITDA saudáveis.
        Endividamento: dívida bruta, dívida líquida, relação dívida líquida/EBITDA e índice de cobertura de juros.
        Eficiência operacional: giro de ativos e margem operacional.
        Rentabilidade: ROE (retorno sobre patrimônio líquido) e ROIC (retorno sobre capital investido).
        Valuation: múltiplos como P/L, P/VPA, EV/EBITDA, dividend yield, Preço Justo (fórmula de Graham) e Preço Topo (método Bazin), baseando-se em projeções conservadoras.
        Outros fatores: evolução do patrimônio líquido, geração de caixa operacional e CAGR de receita, lucro e dividendos.
        A análise deve ser sempre baseada em dados históricos, projeções prudentes e margem de segurança.
        Com base nessa análise, selecione os 5 melhores ativos de cada setor, priorizando aqueles que mostram maior solidez e potencial de valorização,
        mesmo em cenários adversos.
        Para cada ativo selecionado, atribua:
        Uma nota de 0 a 10 para o grau de indicação de investimento, acompanhada da porcentagem estimada de possível assertividade dessa indicação.
        Um ranking de risco em três níveis: Altamente arriscado, Moderado ou Seguro.
        Um ranking de risco versus retorno para auxiliar na compreensão do equilíbrio entre potencial ganho e risco assumido.
        Apresente um relatório detalhado para cada ativo, com análises claras dos indicadores econômicos mencionados, justificando a seleção e o posicionamento dos ativos.
        Se o input for um ativo individual, direcione a análise para ativos do mesmo setor ou segmento, garantindo que a lista contenha apenas os melhores ativos, segundo os critérios estabelecidos.
        Você é um avaliador especialista em análise fundamentalista, micro e macroeconomia.
        Com base em uma abordagem simples para um investidor iniciante, apresente os dados coletados até agora de forma sucinta,
        com linguagem que até um novato entenderia e suas siglas entre parênteses (ex: IPI, IPCA).
        [...]
        No final, deixe claro que não é uma recomendação de investimentos, apenas um exercício acadêmico e que nos isentamos totalmente de qualquer responsabilidade tomada por qualquer indivíduo.
        """,
        description="Agente Avaliador de Investimentos",
    )

    entrada_do_avaliador = f"""
    Tópico: {topico}
    Analise Macro: {analise_macro}
    Analise Micro: {analise_micro}
    Analise Final: {analise_final}
    Analise Fundamentalista: {fundamentalista}
    """

    # Corrigido aqui:
    avaliacao = call_agent(avaliador, entrada_do_avaliador)
    return avaliacao


In [27]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando SAAI (Sistema de Avaliação Avançada de Investimentos) 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o setor ou empresa sobre o qual você gostaria de conhecer sobre investimentos: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o setor ou empresa!")
else:
    print(f"Maravilha! Vamos verificar as melhores possibilidades de investimentos no setor relacionado a {topico}")

    coleta_de_dados = agente_coletor(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Coletor de Dados) ---\n")
    display(to_markdown(coleta_de_dados))
    print("--------------------------------------------------------------")

    analise_macro = agente_analista_macro(topico, coleta_de_dados)
    print("\n--- 📝 Resultado do Agente 2 (Analista Macroeconomico) ---\n")
    display(to_markdown(analise_macro))
    print("--------------------------------------------------------------")

    analise_micro = agente_analista_micro(topico, coleta_de_dados)
    print("\n--- 📝 Resultado do Agente 3 (Analista Microeconomico) ---\n")
    display(to_markdown(analise_micro))
    print("--------------------------------------------------------------")


    analise_final = agente_analista_cenarios(topico, coleta_de_dados, analise_macro, analise_micro)
    print("\n--- 📝 Resultado do Agente 4 (Analista de Cenarios) ---\n")
    display(to_markdown(analise_final))
    print("--------------------------------------------------------------")

    fundamentalista = agente_analista_fundamentalista(topico, analise_macro, analise_micro, analise_final)
    print("\n--- 📝 Resultado do Agente 5 (Analista Fundamentalista) ---\n")
    display(to_markdown(fundamentalista))
    print("--------------------------------------------------------------")

    avaliador = agente_avaliador(topico, analise_macro, analise_micro, analise_final, fundamentalista)
    print("\n--- 📝 Resultado do Agente 6 (Avaliador de Investimentos) ---\n")
    display(to_markdown(avaliador))
    print("--------------------------------------------------------------")

🚀 Iniciando SAAI (Sistema de Avaliação Avançada de Investimentos) 🚀
❓ Por favor, digite o setor ou empresa sobre o qual você gostaria de conhecer sobre investimentos: Ouro
Maravilha! Vamos verificar as melhores possibilidades de investimentos no setor relacionado a Ouro

--- 📝 Resultado do Agente 1 (Coletor de Dados) ---



> Okay, vamos coletar e estruturar uma base de dados de ativos relevantes e correlacionados ao ouro, com dados atualizados e foco em fundamentos sólidos.
> 
> 
> Com base nas informações coletadas, aqui está uma base de dados estruturada de ativos relevantes e correlacionados ao ouro, com foco em fundamentos sólidos e dados atualizados até maio de 2025:
> 
> **Ações Brasileiras**
> 
> Não foram encontradas ações brasileiras diretamente ligadas à exploração de ouro com informações suficientes para análise.
> 
> **Ações Internacionais**
> 
> 1.  **Newmont Corporation (NEM)**:
>     *   **Justificativa:** Maior produtora de ouro do mundo, com ativos em jurisdições de mineração estáveis (EUA, Canadá, Austrália). Forte desempenho ESG. Em 2025, reportou um fluxo de caixa livre recorde de US$ 1,2 bilhão, impulsionado por um aumento de 41% nos preços do ouro realizados em relação ao ano anterior. Apesar de um declínio de 8,3% na produção, as alienações estratégicas da Newmont fortaleceram seu balanço, gerando mais de US$ 2,5 bilhões em receitas em dinheiro.
> 2.  **Barrick Gold Corporation (GOLD)**:
>     *   **Justificativa:** Uma das maiores empresas do setor, reconhecida pela disciplina de gestão e operações eficientes em termos de custos. Reduziu agressivamente suas dívidas, expandiu o fluxo de caixa livre e devolveu capital aos acionistas.
> 3.  **Agnico Eagle Mines Limited (AEM)**:
>     *   **Justificativa:** Operações bem administradas e capacidade de crescer sem depender de grandes aquisições.
> 4.  **Franco-Nevada Corporation (FNV)**:
>     *   **Justificativa:** Não diretamente uma mineradora, mas uma empresa de royalties de mineração, o que oferece uma forma diversificada de exposição ao ouro.
> 5.  **Kinross Gold Corporation (KGC)**:
>     *   **Justificativa:** Grande mineradora de ouro com operações em diversos países.
> 6.  **AngloGold Ashanti (AU)**:
>     *   **Justificativa:** Mineradora global com um portfólio diversificado de minas. Produziu 2.7 milhões de onças de ouro em 2022 e 3.6 milhões de onças de prata. Possui enormes reservas de 162 Moz de ouro (60 anos de produção).
> 7.  **DRDGOLD Limited (DRD)**
> 8.  **Harmony Gold Mining Company Limited (HMY)**
> 9.  **Alamos Gold Inc. (AGI)**
> 10. **First Majestic Silver (AG)**
> 
> *É crucial ressaltar que a Securities.io não fornece consultoria de investimento e recomenda que seja feita uma análise com um profissional antes de qualquer tomada de decisão.*
> 
> **ETFs de Ouro**
> 
> 1.  **VanEck Gold Miners ETF (GDX)**:
>     *   **Justificativa:** Investe nas principais companhias de mineração de ouro. As cinco maiores participações do veículo financeiro são Newmont (NEM), Agnico Eagle Mines (AEM), Wheaton Precious Metals (WPM), Barrick Gold (ABX) e Franco-Nevada (FNV)
> 2.  **abrdn Physical Gold Shares (SGOL)**
>     *   **Justificativa:** ETF de ouro físico
> 3.  **Ishares Gold Trust (IAU)**:
>     *   **Justificativa:** ETF de ouro físico
> 4.  **Investo GLDX11 (Brasil)**:
>     *   **Justificativa:** Lançado recentemente no Brasil, replica o desempenho do GDX.
> 5.  **Axis Gold ETF (Índia)**
> 6.  **IAUM**
> 
> **Commodities**
> 
> 1.  **Prata:**
>     *   **Justificativa:** Tradicionalmente correlacionada ao ouro, a prata pode oferecer oportunidades de valorização.
> 2.  **Cobre:**
>     *   **Justificativa:** Metais básicos podem se beneficiar do aumento da demanda global.
> 
> **Fundos Imobiliários (FIIs)**
> 
> Não foram encontrados fundos imobiliários diretamente atrelados a exploração ou reservas de ouro. No entanto, a diversificação em FIIs de tijolo é sempre recomendada.
> 
> 1. Brasil Plural Absoluto FoF (BPFF11)
> 2. Kinea Renda Imobiliária (KNRI11)
> 3. Kinea Rendimentos Imobiliários (KNCR11)
> 4. MXRF11
> 
> **Investimentos Correlacionados**
> 
> 1.  **Ações de empresas de streaming de ouro (gold streaming companies)**:
>     *   **Justificativa:** Empresas como Wheaton Precious Metals financiam mineradoras em troca de uma parte da produção futura.
> 2.  **Bitcoin:**
>     *   **Justificativa:** Embora volátil, o Bitcoin tem sido considerado por alguns como um "ouro digital".
> 3.  **Títulos do Tesouro indexados à inflação (Treasury Inflation-Protected Securities - TIPS):**
>     *   **Justificativa:** Protegem contra a inflação, que historicamente impulsiona o preço do ouro.
> 4.  **Moedas de países com grandes reservas de ouro (dólar australiano, rand sul-africano):**
>     *   **Justificativa:** Podem se valorizar em momentos de alta do ouro.
> 5. **CDBs:**
>     *   **Justificativa:** Diversificação de baixo risco.
> 6. **Títulos Públicos Federais:**
>     *   **Justificativa:** Segurança e estabilidade.
> 7. **Letras:**
>     *   **Justificativa:** Opções de investimento em renda fixa.
> 
> **Disclaimer:** É importante notar que este não é uma recomendação de investimento. As condições de mercado estão sujeitas a mudanças, e é essencial que você procure orientação de um profissional financeiro antes de tomar qualquer decisão de investimento.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Analista Macroeconomico) ---



> Okay, com base nos ativos listados, farei uma análise macroeconômica relacionando-os ao cenário atual e possíveis desdobramentos, juntamente com notícias relevantes.
> 
> **Análise Macroeconômica e Ativos Correlacionados ao Ouro**
> 
> O ouro tradicionalmente serve como um porto seguro em tempos de incerteza econômica e inflação. Atualmente, vários fatores macroeconômicos influenciam o desempenho do ouro e seus ativos relacionados:
> 
> *   **Inflação:** A inflação global persistente, embora com sinais de arrefecimento em algumas regiões, continua a ser um fator chave. Bancos centrais, como o Federal Reserve (Fed) dos EUA, têm adotado políticas monetárias restritivas para combater a inflação, elevando as taxas de juros. Juros mais altos tendem a aumentar o custo de oportunidade de manter ouro (que não rende juros), o que pode pressionar os preços. No entanto, se a inflação se mostrar mais persistente do que o esperado, o ouro pode se beneficiar como proteção.
> *   **Taxas de Juros:** O aumento das taxas de juros pelo Fed e outros bancos centrais tem um impacto complexo. Por um lado, juros mais altos tornam o ouro menos atraente em comparação com ativos que rendem juros. Por outro lado, se o aperto monetário levar a uma desaceleração econômica ou recessão, o ouro pode se beneficiar como um ativo de refúgio.
> *   **Crescimento Econômico:** A perspectiva de crescimento econômico global é incerta. Dados recentes apontam para uma desaceleração em várias economias importantes. Uma recessão global poderia impulsionar a demanda por ouro como um ativo seguro.
> *   **Geopolítica:** Tensões geopolíticas, como conflitos e incertezas políticas, geralmente aumentam a demanda por ouro.
> 
> **Análise dos Ativos**
> 
> **Ações Internacionais**
> 
> *   **Newmont Corporation (NEM), Barrick Gold Corporation (GOLD), Agnico Eagle Mines Limited (AEM), Franco-Nevada Corporation (FNV), Kinross Gold Corporation (KGC), AngloGold Ashanti (AU), DRDGOLD Limited (DRD), Harmony Gold Mining Company Limited (HMY), Alamos Gold Inc. (AGI)**: O desempenho dessas empresas está diretamente ligado ao preço do ouro, custos de produção e eficiência operacional. Um ambiente de inflação elevada pode aumentar os custos de produção, mas preços mais altos do ouro podem compensar. A gestão da dívida e a capacidade de gerar fluxo de caixa livre são fatores importantes para avaliar essas empresas.
> *   **First Majestic Silver (AG)**: Além do preço da prata, fatores específicos da empresa, como custos de produção e desenvolvimento de projetos, também influenciam o desempenho.
> 
> **ETFs de Ouro**
> 
> *   **VanEck Gold Miners ETF (GDX), abrdn Physical Gold Shares (SGOL), Ishares Gold Trust (IAU), Investo GLDX11 (Brasil), Axis Gold ETF (Índia), IAUM:** ETFs que investem em ouro físico ou em empresas de mineração de ouro replicam o desempenho do ouro. A demanda por esses ETFs pode aumentar em tempos de incerteza.
> 
> **Commodities**
> 
> *   **Prata:** A prata é influenciada tanto por fatores de metais preciosos quanto por sua demanda industrial. A recuperação econômica global pode impulsionar a demanda por prata em aplicações industriais.
> *   **Cobre:** O cobre é um metal industrial que se beneficia do crescimento econômico global, especialmente da demanda por infraestrutura e construção.
> 
> **Investimentos Correlacionados**
> 
> *   **Ações de empresas de streaming de ouro:** Oferecem uma forma diversificada de exposição ao ouro, pois seus fluxos de receita estão ligados à produção de várias minas.
> *   **Bitcoin:** A correlação entre Bitcoin e ouro tem variado. Em alguns momentos, o Bitcoin tem se comportado como um ativo de refúgio, mas sua volatilidade é muito maior do que a do ouro.
> *   **Títulos do Tesouro indexados à inflação (TIPS):** Protegem contra a inflação, o que pode reduzir o apelo do ouro como proteção contra a inflação.
> *   **Moedas de países com grandes reservas de ouro (dólar australiano, rand sul-africano):** Podem se valorizar em momentos de alta do ouro, mas também são influenciadas por outros fatores econômicos específicos de cada país.
> 
> **Notícias Macroeconômicas Relevantes**
> 
> 1.  **Inflação nos EUA:** "Inflação nos EUA desacelera em abril, mas ainda preocupa o Fed. O Índice de Preços ao Consumidor (CPI) subiu 0,3% em abril, mostrando uma leve desaceleração em relação ao mês anterior. No entanto, a inflação ainda está acima da meta de 2% do Fed, o que pode levar o banco central a manter as taxas de juros elevadas por mais tempo." Isso afeta diretamente o custo de oportunidade de manter ouro.
> 
> 
> 2.  **Perspectivas de Crescimento Global:** "OCDE reduz previsão de PIB global para 3,1% em 2025 devido a incertezas geopolíticas e aumento de barreiras comerciais." Uma desaceleração no crescimento global pode aumentar a demanda por ativos de refúgio, como o ouro.
> 
> Essas notícias macroeconômicas podem influenciar significativamente o comportamento dos ativos listados. Por exemplo:
> 
> *   **Ações de mineradoras de ouro:** Serão impactadas positivamente por um aumento no preço do ouro, mas negativamente por custos de produção mais altos devido à inflação.
> *   **ETFs de ouro:** Replicarão o desempenho do ouro, se beneficiando de um ambiente de inflação persistente ou incerteza econômica.
> *   **Prata e cobre:** Serão influenciados pelas perspectivas de crescimento global, com a prata também se beneficiando de seu status como metal precioso.
> 
> **Recomendações Gerais:**
> 
> *   Acompanhar de perto os dados de inflação e as decisões de política monetária dos bancos centrais.
> *   Monitorar os desenvolvimentos geopolíticos e seu impacto no sentimento do mercado.
> *   Considerar a diversificação entre diferentes classes de ativos para mitigar riscos.
> 
> Lembre-se que esta análise é baseada em informações disponíveis até maio de 2025 e o cenário macroeconômico pode mudar rapidamente.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Analista Microeconomico) ---



...

--------------------------------------------------------------

--- 📝 Resultado do Agente 5 (Analista Fundamentalista) ---



> Com base nas análises macro e microeconômicas, nas projeções de cenários e nas observações estratégicas, apresento uma seleção dos cinco ativos mais sólidos e promissores relacionados ao ouro, juntamente com uma avaliação do seu potencial de valorização e riscos associados:
> 
> **Top 5 Ativos Relacionados ao Ouro**
> 
> 1.  **abrdn Physical Gold Shares (SGOL) e Ishares Gold Trust (IAU):**
> 
>     *   **Justificativa:** Estes ETFs de ouro físico oferecem uma exposição direta e líquida ao preço do ouro, sem a necessidade de armazenar o metal físico. Em um cenário de inflação persistente, incertezas geopolíticas e volatilidade econômica, o ouro tende a se valorizar como um ativo de refúgio, impulsionando o desempenho desses ETFs.
>     *   **Potencial de Valorização:** Historicamente, o ouro tem demonstrado um desempenho superior em períodos de crise e inflação elevada. Projeta-se que o ouro possa superar o IPCA, o S&P 500 e a Nasdaq em cenários de estagflação ou recessão moderada.
>     *   **Riscos:** A elevação das taxas de juros pode aumentar o custo de oportunidade de manter ouro, o que pode pressionar os preços. Além disso, uma forte recuperação econômica global pode reduzir a demanda por ativos de refúgio.
>     *   **Observação:** Monitore os fluxos de entrada e saída dos ETFs de ouro, pois eles podem indicar o sentimento do mercado em relação ao ouro.
> 
> 2.  **Newmont Corporation (NEM):**
> 
>     *   **Justificativa:** A Newmont é a maior produtora de ouro do mundo, com operações diversificadas e um histórico comprovado de geração de fluxo de caixa livre. A empresa se beneficia de economias de escala e de uma presença global, o que a torna mais resiliente a choques econômicos e operacionais.
>     *   **Potencial de Valorização:** Em um cenário de alta dos preços do ouro, a Newmont tem a capacidade de aumentar sua produção e rentabilidade, impulsionando o valor de suas ações. A empresa também se destaca pelo forte desempenho em critérios ESG, o que pode atrair investidores socialmente responsáveis.
>     *   **Riscos:** Os custos de produção mais altos devido à inflação podem reduzir a rentabilidade da Newmont. Além disso, interrupções na produção, flutuações cambiais e riscos políticos podem afetar negativamente o desempenho da empresa.
>     *   **Observação:** Acompanhe os custos de produção da Newmont, sua capacidade de gerenciar a dívida e seus projetos de expansão.
> 
> 3.  **Barrick Gold Corporation (GOLD):**
> 
>     *   **Justificativa:** A Barrick Gold é reconhecida por sua disciplina de gestão, operações eficientes em termos de custos e foco na redução de dívidas. A empresa possui um portfólio diversificado de minas de ouro e cobre, o que a torna menos dependente do desempenho de um único ativo.
>     *   **Potencial de Valorização:** A Barrick Gold tem um forte potencial de valorização em um cenário de alta dos preços do ouro e do cobre. A empresa também se beneficia de sua capacidade de gerar fluxo de caixa livre robusto e de sua gestão proativa de riscos.
>     *   **Riscos:** A Barrick Gold está exposta aos riscos inerentes à mineração, como interrupções na produção, flutuações cambiais e riscos políticos. Além disso, a empresa pode enfrentar desafios na gestão de seus projetos de expansão.
>     *   **Observação:** Monitore a capacidade da Barrick Gold de reduzir dívidas, expandir o fluxo de caixa livre e gerenciar seus projetos de expansão.
> 
> 4.  **Prata:**
> 
>     *   **Justificativa:** A prata é um metal precioso com aplicações industriais diversificadas, o que a torna menos dependente do desempenho do ouro. A demanda por prata tem aumentado em setores como eletrônicos, energia solar e veículos elétricos, o que pode impulsionar seu preço.
>     *   **Potencial de Valorização:** Em um cenário de crescimento econômico global e inflação persistente, a prata tem um forte potencial de valorização. A prata também pode se beneficiar de seu status como um ativo de refúgio em tempos de incerteza.
>     *   **Riscos:** A prata é mais volátil do que o ouro e pode sofrer correções de preço significativas. Além disso, a demanda por prata pode ser afetada por uma desaceleração econômica global ou por mudanças tecnológicas.
>     *   **Observação:** Monitore os déficits no mercado de prata, a demanda industrial e os estoques globais.
> 
> 5.  **Títulos do Tesouro Indexados à Inflação (TIPS):**
> 
>     *   **Justificativa:** Os TIPS oferecem proteção contra a inflação, ajustando seu valor principal com base nas mudanças no Índice de Preços ao Consumidor (CPI). Em um cenário de inflação persistente, os TIPS podem preservar o poder de compra do investidor e oferecer um retorno real acima da inflação.
>     *   **Potencial de Valorização:** Os TIPS têm um potencial de valorização limitado em comparação com outros ativos, mas oferecem um refúgio seguro em tempos de inflação elevada. Além disso, os TIPS podem se beneficiar de uma queda nas taxas de juros reais.
>     *   **Riscos:** Os TIPS estão sujeitos ao risco de taxa de juros e podem perder valor se as taxas de juros subirem. Além disso, os TIPS podem ter um desempenho inferior se a deflação se tornar uma preocupação.
>     *   **Observação:** Acompanhe as expectativas de inflação do mercado, as taxas de juros reais e as projeções do CPI.
> 
> **Recomendações Finais**
> 
> *   **Diversificação:** É fundamental diversificar a carteira de investimentos, alocando uma parte dos recursos em ativos relacionados ao ouro e em outras classes de ativos, como ações, títulos e imóveis.
> *   **Alocação Estratégica:** A alocação de recursos em cada ativo deve ser baseada no perfil de risco, objetivos de investimento e horizonte de tempo de cada investidor.
> *   **Gestão Ativa:** É importante acompanhar de perto os desenvolvimentos macroeconômicos e microeconômicos, ajustando a estratégia de investimento conforme necessário.
> *   **Visão de Longo Prazo:** O investimento em ouro e ativos relacionados deve ser encarado como uma estratégia de longo prazo, visando proteger o patrimônio e se beneficiar de incertezas econômicas.
> *   **Consultoria Profissional:** Recomenda-se buscar a orientação de um consultor financeiro qualificado para auxiliar na tomada de decisões de investimento.
> 
> **Observações Importantes**
> 
> *   As análises e recomendações apresentadas são baseadas em informações disponíveis até a presente data e estão sujeitas a mudanças sem aviso prévio.
> *   O desempenho passado não é garantia de resultados futuros.
> *   O investimento em ativos financeiros envolve riscos e pode resultar em perdas.
> *   Cada investidor deve realizar sua própria análise e tomar suas decisões de investimento com base em suas circunstâncias individuais.
> 
> Espero que esta análise detalhada e abrangente seja útil para auxiliar na tomada de decisões de investimento relacionadas ao ouro.


--------------------------------------------------------------

--- 📝 Resultado do Agente 6 (Avaliador de Investimentos) ---



> Okay, com base nas análises macro e microeconômicas, nas projeções de cenários e nas observações estratégicas, apresento uma seleção dos cinco ativos mais sólidos e promissores relacionados ao ouro, juntamente com uma avaliação do seu potencial de valorização e riscos associados:
> 
> **Top 5 Ativos Relacionados ao Ouro**
> 
> 1.  **abrdn Physical Gold Shares (SGOL) e Ishares Gold Trust (IAU):**
> 
>     *   **O que são:** São como "caixinhas" (ETFs) que representam ouro físico de verdade.
>     *   **Por que são bons:** Quando as coisas ficam incertas (inflação alta, guerras), o ouro tende a subir, e essas "caixinhas" sobem junto.
>     *   **Risco:** Se os juros dos bancos subirem muito, pode ser que o ouro fique menos interessante.
>     *   **Nota:** 9/10
>     *   **Assertividade:** 85%
>     *   **Risco:** Seguro
>     *   **Risco vs Retorno:** Bom
> 
> 2.  **Newmont Corporation (NEM):**
> 
>     *   **O que é:** A maior empresa do mundo que extrai ouro.
>     *   **Por que é boa:** Ela é grande, organizada e consegue ganhar dinheiro mesmo quando as coisas não estão fáceis.
>     *   **Risco:** Se os custos para tirar o ouro da terra aumentarem, pode ser que ela ganhe menos dinheiro.
>     *   **Nota:** 8/10
>     *   **Assertividade:** 80%
>     *   **Risco:** Moderado
>     *   **Risco vs Retorno:** Bom
> 
> 3.  **Barrick Gold Corporation (GOLD):**
> 
>     *   **O que é:** Outra empresa grande que extrai ouro.
>     *   **Por que é boa:** Ela é boa em controlar os gastos e tem várias minas diferentes.
>     *   **Risco:** Problemas nas minas ou mudanças nas leis dos países onde ela atua podem atrapalhar.
>     *   **Nota:** 7/10
>     *   **Assertividade:** 75%
>     *   **Risco:** Moderado
>     *   **Risco vs Retorno:** Razoável
> 
> 4.  **Prata:**
> 
>     *   **O que é:** Um metal que vale dinheiro e também é usado em várias coisas, como eletrônicos.
>     *   **Por que é boa:** Se a economia mundial crescer, a prata pode valer mais.
>     *   **Risco:** A prata pode ser mais "nervosa" (volátil) que o ouro.
>     *   **Nota:** 6/10
>     *   **Assertividade:** 70%
>     *   **Risco:** Moderado
>     *   **Risco vs Retorno:** Razoável
> 
> 5.  **Títulos do Tesouro Indexados à Inflação (TIPS):**
> 
>     *   **O que são:** Títulos do governo que protegem o seu dinheiro da inflação (IPCA).
>     *   **Por que são bons:** Se a inflação subir, o valor desses títulos também sobe.
>     *   **Risco:** Se os juros subirem, o valor desses títulos pode cair.
>     *   **Nota:** 5/10
>     *   **Assertividade:** 65%
>     *   **Risco:** Seguro
>     *   **Risco vs Retorno:** Razoável
> 
> **Resumo para Iniciantes:**
> 
> *   **Ouro é um "porto seguro":** Em tempos de crise, as pessoas correm para o ouro, o que faz o preço subir.
> *   **Inflação (IPCA):** É o aumento dos preços. Se a inflação sobe, o ouro também pode subir.
> *   **Juros:** Se os juros sobem, investir em ouro pode ficar menos interessante.
> *   **ETFs:** São como "caixinhas" que facilitam investir em ouro sem precisar comprar o metal físico.
> 
> **Observações:**
> 
> *   **Diversifique:** Não coloque todo o seu dinheiro em um só lugar.
> *   **Acompanhe:** Fique de olho nas notícias e veja como a economia está se comportando.
> *   **Longo prazo:** Pense em investir em ouro como uma forma de proteger o seu dinheiro ao longo do tempo.
> 
> **Atenção:**
> 
> *   **Não é recomendação:** Isso não é uma recomendação de investimento. É apenas um exercício para entender melhor o mercado.
> *   **Sua decisão:** A decisão de investir é sua e você é o único responsável por ela.
> *   **Sem garantia:** Ninguém pode garantir que você vai ganhar dinheiro investindo.
> 
> **Isenção de responsabilidade:**
> 
> Este relatório é apenas para fins educacionais e informativos. Não somos responsáveis por quaisquer decisões de investimento tomadas com base nessas informações. Consulte um profissional financeiro antes de investir.
> 


--------------------------------------------------------------
